# Projet Data Visualisation
## Transformation des données pour leur utilisation

In [43]:
import pandas as pd
import re

### Lecture des données

In [44]:
file = "releves_bancaires_2021.csv"
df_operations = pd.read_csv(f"../data/{file}", sep = ";")[["Libellé", "Date", "Débit euros", "Catégorie"]]
df_operations = df_operations.loc[df_operations["Débit euros"].notnull ()]

In [45]:
df_operations.head()

,Libellé,Date,Débit euros,Catégorie
0,PAIEMENT PAR CARTE \nREST UNIV PUVIS 69 V...,17/12/2021,"6,6",Cantine
1,PAIEMENT PAR CARTE \nMAXICOFFEE ARA NEYRO...,16/12/2021,"2,3",Snacking
2,PAIEMENT PAR CARTE \nMAXICOFFEE ARA NEYRO...,16/12/2021,"1,6",Snacking
3,PAIEMENT PAR CARTE \nMAXICOFFEE ARA NEYRO...,16/12/2021,1,Snacking
4,PAIEMENT PAR CARTE \nSEAZEN LYON 7EME ...,15/12/2021,"6,9",Restaurants


In [46]:
df_expenses_cat = pd.read_excel("../data/expenses_by_category.xlsx").rename(columns={"Unnamed: 0": "Catégorie", "Autres inactifs":"Etudiants", "Ensemble":"Français"})[["Catégorie", "Etudiants", "Français"]][:-1]

In [47]:
df_expenses_cat.head()

,Catégorie,Etudiants,Français
0,Produits alimentaires et boissons non alcoolisées,16.768891,16.090613
1,Boissons alcoolisées et tabac,3.517955,2.875439
2,Articles d'habillement et chaussures,5.812929,5.012124
3,"Logement, eau, gaz, électricité et autres comb...",24.132486,16.251477
4,"Meubles, articles de ménage et entretien coura...",4.751010,5.684839


### Comparaison et mapping des catégories

In [48]:
df_operations["Catégorie"].unique()

array(['Cantine', 'Snacking', 'Restaurants', 'Alimentation', 'Livres',
       'Transports', 'Mutuelle de santé', 'Téléphone', 'Loyer', 'Cadeaux',
       'Travaux', 'Pharmacie', 'Jeux', 'Multimédia', 'Évènements',
       'Habillement', 'Autres dépenses', 'Sorties', 'Frais bancaires',
       'Ameublement/équipement', 'Santé/Bien être', "Retrait d'argent",
       'Dons', 'Chaussures', 'Frais de scolarité', 'Assurance logement',
       'Frais immobiliers'], dtype=object)

In [49]:
df_expenses_cat["Catégorie"].unique()

array(['Produits alimentaires et boissons non alcoolisées',
       'Boissons alcoolisées et tabac',
       "Articles d'habillement et chaussures",
       'Logement, eau, gaz, électricité et autres combustibles',
       'Meubles, articles de ménage et entretien courant de la maison',
       'Santé', 'Transports', 'Communications', 'Loisirs et culture',
       'Enseignement', 'Restauration et hôtels',
       'Biens et services divers (1)'], dtype=object)

In [50]:
categories_mapping_operations = {
    "Habitation":["Habitation", "Travaux", "Electricité, gaz, eau", "Impôts : taxe foncière", "Impôts : taxe habitation",
                 "Habitation secondaire", "Loyer", "Jardin", "Assurance logement", "Ménage/repassage",
                 "Ameublement/équipement"],
    "Loisirs":["Loisirs", "Multimédia", "Vacances", "Animaux", "Sport", "Sorties", "Cadeaux", "Evènements", "Jeux",
              "Hôtels", "Livres", "Restaurants"],
    "Communications":["Téléphone", "Internet/TV", "Communications"],
    "Alimentation" :["Alimentation", "Cantine", "Snacking"],
    "Transports":["Transports", "Véhicule", "Moto", "Péages", "Essence", "Location Voiture", "Assurance auto",
                 "Assurance moto", "Parking"],
    "Santé/Bien être":["Santé/Bien être", "Soin du corps", "Services à la personne", "Mutuelle de santé", "Pharmacie",
                      "Assurance santé", "Aides à domicile", "Coiffeur"],
    "Habillement":["Habillement", "Chaussures", "Accessoires"],
    "Autres":["Frais Pro", "Enfants", "Amendes", "Frais de justice", "Frais de scolarité", "Gardes d'enfant", "Dons", 
             "Impôts : autres", "Assurances : autres", "Autres dépenses",
             "Impôts : revenus", "Epargne", "Prêts", "Agios", "Frais bancaires", "Assurance vie", "Bourse", 
             "Retrait d'argent", "Prêt relais"]
}
categories_mapping_operations = {subcat:cat for cat in categories_mapping_operations.keys() for subcat in categories_mapping_operations[cat]}

In [51]:
categories_mapping_expenses_cat = {
    'Produits alimentaires et boissons non alcoolisées':"Alimentation",
    'Boissons alcoolisées et tabac':"Alimentation",
    "Articles d'habillement et chaussures":"Habillement",
    'Logement, eau, gaz, électricité et autres combustibles':"Habitation",
    'Meubles, articles de ménage et entretien courant de la maison' :"Habitation",
    'Santé' : "Santé/Bien être",
    'Transports':"Transports",
    'Communications': "Communications",
    'Loisirs et culture':"Loisirs",
    'Enseignement':"Autres",
    'Restauration et hôtels':"Loisirs",
    'Biens et services divers (1)':"Autres"
}

### Transformation des données

In [52]:
df_operations = df_operations.rename(columns = {"Débit euros": "Montant"})
df_operations["Libellé"] = df_operations["Libellé"].map (lambda x: re.split ("\d+", x.splitlines ()[1])[0])

In [53]:
df_operations["Catégorie"] = df_operations["Catégorie"].map(categories_mapping_operations)
df_operations.head()

,Libellé,Date,Montant,Catégorie
0,REST UNIV PUVIS,17/12/2021,"6,6",Alimentation
1,MAXICOFFEE ARA NEYRON,16/12/2021,"2,3",Alimentation
2,MAXICOFFEE ARA NEYRON,16/12/2021,"1,6",Alimentation
3,MAXICOFFEE ARA NEYRON,16/12/2021,1,Alimentation
4,SEAZEN LYON,15/12/2021,"6,9",Loisirs


In [54]:
df_operations["Catégorie"].value_counts()

Alimentation       130
Loisirs             95
Santé/Bien être     40
Transports          35
Habitation          17
Communications      12
Autres              12
Habillement          8
Name: Catégorie, dtype: int64

In [55]:
df_expenses_cat["Catégorie"] = df_expenses_cat["Catégorie"].map(categories_mapping_expenses_cat)
df_expenses_cat = df_expenses_cat.groupby("Catégorie").sum().reset_index()
df_expenses_cat.head()

,Catégorie,Etudiants,Français
0,Alimentation,20.286845,18.966052
1,Autres,14.800799,16.625232
2,Communications,3.649845,2.711766
3,Habillement,5.812929,5.012124
4,Habitation,28.883495,21.936316


### Sauvegarde des données

In [56]:
df_operations.to_csv("../data/operations.csv", index=False)
df_expenses_cat.to_csv("../data/repartition_categories.csv", index=False)